In [ ]:
import ccxt, pytz
import pandas as pd
from datetime import datetime
import time

# It's a good practice to not hard-code sensitive information in the script.
# Consider using environment variables for API keys and secrets.
api_key = "XXX"
api_secret = "XXX"
api_passphrase = 'XXX'
# Define the trading pair and timeframe
symbol = 'SOL/USDT'
timeframe = '1h'  # 1-hour candlesticks

# Initialize the KuCoin client with API credentials
kucoin = ccxt.kucoin({
    'apiKey': api_key,
    'secret': api_secret,
    'password': api_passphrase,
})

start_date = datetime(2023, 1, 1)  # Define your start date
start_timestamp = int(start_date.timestamp() * 1000)  # Convert to milliseconds

# Fetch the current timestamp in milliseconds
current_timestamp = int(datetime.now().timestamp() * 1000)

historical_data = []

try:
    while start_timestamp < current_timestamp:
        # Fetch data
        print(f"Fetching data since {datetime.fromtimestamp(start_timestamp / 1000)}")
        data = kucoin.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        
        if not data:
            print("No more data to fetch.")
            break
        
        # Extend the historical data with new chunk
        historical_data.extend(data)
        
        # Update start_timestamp to the last fetched candle's timestamp plus one interval
        start_timestamp = data[-1][0] + 60*60*1000  # Assuming 1h timeframe, adjust for other timeframes
        
        # Respect the rate limit
        time.sleep(kucoin.rateLimit / 1000)
        
except ccxt.BaseError as e:
    print(f"An error occurred: {e}")

# Columns for the DataFrame
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

# Convert the historical data into a DataFrame
df = pd.DataFrame(historical_data, columns=columns)



# Convert the timestamp from milliseconds to a readable datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Convert to Toronto Timezone
toronto_tz = pytz.timezone('America/Toronto')
df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(toronto_tz)

# Continue with your code to save the DataFrame to a CSV file
csv_filename = 'historical_data_sol.csv'
df.to_csv(csv_filename, index=False)

print(f"Historical data successfully saved to {csv_filename}, with timestamps converted to Toronto time.")
